In [10]:
# for text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# for topic modeling
from gensim import corpora
from gensim.models import LdaModel

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nafiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nafiz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nafiz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
documents = [
    "Rafeal Nadal Joins Roger Federer in Missing U.S Open",
    "Rafeal Nadal Is Out of the Auutralian Open",
    "Biden Announces Virus Measures",
    "Biden's Virus Plans Meet Reality",
    "Where Biden's Virus Plan Stands"
]

In [12]:
stop_words = set(stopwords.words('english'))  # Create a set of English stopwords
lemmatizer = WordNetLemmatizer()              # Initialize a WordNet Lemmatizer

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()]  # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords from the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize each token
    return tokens  # Return the preprocessed tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]  # Preprocess each document in the list
preprocessed_documents


[['rafeal', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafeal', 'nadal', 'auutralian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

In [14]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]


In [18]:
# corpus: bag-of-words representation of the documents
# num_topics: number of topics to be extracted by the model
# id2word=dictionary: dictionary mapping from word IDs to words
# passes: number of passes through the corpus during training

# Train an LDA model on the corpus with 2 topics using Gensim's LdaModel class
lda_model = LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=15)



In [20]:
# empty list to store dominant topic labels for each document
article_labels = []

#iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # for each document, convert to bow representation
    bow = dictionary.doc2bow(doc)
    # get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # append to the list
    article_labels.append(dominant_topic)

In [24]:
import pandas as pd

# Create DataFrame
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df)
print()


Table with Articles and Topic:
                                             Article  Topic
0  Rafeal Nadal Joins Roger Federer in Missing U....      1
1         Rafeal Nadal Is Out of the Auutralian Open      1
2                     Biden Announces Virus Measures      0
3                   Biden's Virus Plans Meet Reality      0
4                    Where Biden's Virus Plan Stands      0



In [26]:
# Print the top terms for each topic
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
- "biden" (weight: 0.166)
- "virus" (weight: 0.166)
- "plan" (weight: 0.119)
- "stand" (weight: 0.071)
- "meet" (weight: 0.071)
- "reality" (weight: 0.071)
- "measure" (weight: 0.071)
- "announces" (weight: 0.071)
- "auutralian" (weight: 0.024)
- "nadal" (weight: 0.024)

Topic 1:
- "open" (weight: 0.131)
- "rafeal" (weight: 0.131)
- "nadal" (weight: 0.131)
- "join" (weight: 0.079)
- "missing" (weight: 0.079)
- "roger" (weight: 0.079)
- "federer" (weight: 0.079)
- "auutralian" (weight: 0.079)
- "virus" (weight: 0.027)
- "biden" (weight: 0.027)

